In [1]:
from sklearn.decomposition import KernelPCA
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, LeakyReLU
from keras.optimizers import Adam

In [2]:
path = "./UJIIndoorLoc/children_13/sorted/00_sorted.csv"
train_df = pd.read_csv(path,header=0)
print(train_df.shape)
a = train_df.mean()
for i in train_df.columns[:520]:
    if a[i]==-110:
        del train_df[i]
print(train_df.shape)
train_df["REF"] = pd.factorize(train_df["REF"])[0].astype(int)#将标签映射到顺序数字上
labels = train_df.REF.values
features = train_df.drop(columns=['TIMESTAMP','PHONEID','USERID','RELATIVEPOSITION',
                                'SPACEID','BUILDINGID','FLOOR','LATITUDE','LONGITUDE',
                                'BF','REF']).values
features,labels,features.shape

(1059, 531)
(1059, 130)


(array([[-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110],
        ...,
        [-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110]], dtype=int64),
 array([ 0,  0,  0, ..., 53, 53, 53]),
 (1059, 119))

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(features, 
                                                    labels, 
                                                    test_size=0.3,
                                                    random_state= 150,
                                                    #random_state：可以理解为随机数种子，主要是为了复现结果而设置
                                                    stratify=labels)
X_train,Y_train = shuffle(X_train, Y_train)

In [4]:
scale = StandardScaler().fit(X_train)
X_train_transform = scale.transform(X_train)
X_test_transform = scale.transform(X_test)
Y_train_encoded = to_categorical(Y_train)
Y_test_encoded = to_categorical(Y_test)

In [5]:
rbf_pca=KernelPCA(n_components = 150,kernel='sigmoid',gamma=0.041111).fit(X_train_transform)
X_train_pca = rbf_pca.transform(X_train_transform)
X_test_pca = rbf_pca.transform(X_test_transform)
X_train_pca,X_test_pca,X_train_pca.shape,X_test_pca.shape

(array([[-6.42884551e-01,  4.37115372e-01,  2.74222617e-02, ...,
         -3.35453323e-04,  2.61862197e-04, -3.95457180e-05],
        [-6.45507116e-01,  2.83826602e-01,  7.47224562e-02, ...,
         -2.50933897e-04, -2.40821825e-03, -2.69425299e-03],
        [ 4.33174112e-01, -1.68877680e-02, -1.20335046e-02, ...,
         -6.98020531e-03, -4.55518269e-03, -4.55869492e-03],
        ...,
        [ 9.36897651e-01, -4.18281660e-02,  1.18618934e-01, ...,
         -9.88119253e-03, -3.98219481e-03,  5.02322942e-03],
        [-5.61415483e-01, -1.59532503e-01, -1.77507860e-01, ...,
         -3.82933504e-03,  1.33121301e-02, -4.94852825e-04],
        [-4.01136700e-01, -9.68631879e-02, -1.29671583e-01, ...,
         -1.12051705e-03,  5.02207496e-03,  7.54278049e-04]]),
 array([[-0.42468489, -0.05854455, -0.08365404, ...,  0.00164905,
          0.00778629, -0.00238198],
        [-0.76897189,  0.31477847,  0.13266429, ..., -0.00266373,
          0.00410417,  0.00475712],
        [-0.55703471, -0.

### 找出kpca最合适的kernel和gama

In [6]:
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline
clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression())
])
param_grid = [{
        "kpca__gamma": np.linspace(0.01, 0.05,10),
        "kpca__kernel": ["rbf", "sigmoid","cosine","poly"]#不能加上"precomputed"，不然会报错 error: X should be a square kernel matrix
    }]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X_train_transform, Y_train)

C:\Users\xfp\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\xfp\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kpca', KernelPCA(n_components=2)),
                                       ('log_reg', LogisticRegression())]),
             param_grid=[{'kpca__gamma': array([0.01      , 0.01444444, 0.01888889, 0.02333333, 0.02777778,
       0.03222222, 0.03666667, 0.04111111, 0.04555556, 0.05      ]),
                          'kpca__kernel': ['rbf', 'sigmoid', 'cosine',
                                           'poly']}])

In [7]:
print(grid_search.best_params_)

{'kpca__gamma': 0.05, 'kpca__kernel': 'sigmoid'}


### train

In [9]:
model = Sequential()
model.add(Dense(256, input_shape=(150,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(54, activation='softmax'))
model.compile(optimizer=Adam(0.0002, 0.5), loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train_pca,Y_train_encoded,epochs=250,batch_size=32,validation_split=0.2,verbose=1)

Epoch 1/250
19/19 [==============================] - 1s 23ms/step - loss: 3.9835 - accuracy: 0.0220 - val_loss: 3.9771 - val_accuracy: 0.0604
Epoch 2/250
19/19 [==============================] - 0s 6ms/step - loss: 3.9686 - accuracy: 0.0456 - val_loss: 3.9652 - val_accuracy: 0.1007
Epoch 3/250
19/19 [==============================] - 0s 5ms/step - loss: 3.9501 - accuracy: 0.0794 - val_loss: 3.9470 - val_accuracy: 0.0738
Epoch 4/250
19/19 [==============================] - 0s 6ms/step - loss: 3.9225 - accuracy: 0.1014 - val_loss: 3.9209 - val_accuracy: 0.0470
Epoch 5/250
19/19 [==============================] - 0s 6ms/step - loss: 3.8836 - accuracy: 0.1233 - val_loss: 3.8824 - val_accuracy: 0.0671
Epoch 6/250
19/19 [==============================] - 0s 6ms/step - loss: 3.8291 - accuracy: 0.1470 - val_loss: 3.8282 - val_accuracy: 0.0738
Epoch 7/250
19/19 [==============================] - 0s 6ms/step - loss: 3.7544 - accuracy: 0.1520 - val_loss: 3.7563 - val_accuracy: 0.0940
Epoch 8/250


Epoch 59/250
19/19 [==============================] - 0s 5ms/step - loss: 0.8183 - accuracy: 0.7990 - val_loss: 1.4144 - val_accuracy: 0.5570
Epoch 60/250
19/19 [==============================] - 0s 5ms/step - loss: 0.8070 - accuracy: 0.8041 - val_loss: 1.4049 - val_accuracy: 0.5570
Epoch 61/250
19/19 [==============================] - 0s 5ms/step - loss: 0.7889 - accuracy: 0.8057 - val_loss: 1.4021 - val_accuracy: 0.5570
Epoch 62/250
19/19 [==============================] - 0s 4ms/step - loss: 0.7767 - accuracy: 0.8024 - val_loss: 1.3875 - val_accuracy: 0.5772
Epoch 63/250
19/19 [==============================] - 0s 4ms/step - loss: 0.7617 - accuracy: 0.8057 - val_loss: 1.3783 - val_accuracy: 0.5638
Epoch 64/250
19/19 [==============================] - 0s 4ms/step - loss: 0.7451 - accuracy: 0.8176 - val_loss: 1.3882 - val_accuracy: 0.5638
Epoch 65/250
19/19 [==============================] - 0s 4ms/step - loss: 0.7342 - accuracy: 0.8193 - val_loss: 1.3694 - val_accuracy: 0.5839
Epoch 

19/19 [==============================] - 0s 6ms/step - loss: 0.3417 - accuracy: 0.9240 - val_loss: 1.3124 - val_accuracy: 0.5973
Epoch 117/250
19/19 [==============================] - 0s 5ms/step - loss: 0.3352 - accuracy: 0.9223 - val_loss: 1.3085 - val_accuracy: 0.5906
Epoch 118/250
19/19 [==============================] - 0s 6ms/step - loss: 0.3319 - accuracy: 0.9274 - val_loss: 1.3155 - val_accuracy: 0.5906
Epoch 119/250
19/19 [==============================] - 0s 7ms/step - loss: 0.3273 - accuracy: 0.9257 - val_loss: 1.3106 - val_accuracy: 0.5973
Epoch 120/250
19/19 [==============================] - 0s 6ms/step - loss: 0.3243 - accuracy: 0.9274 - val_loss: 1.3191 - val_accuracy: 0.5973
Epoch 121/250
19/19 [==============================] - 0s 5ms/step - loss: 0.3209 - accuracy: 0.9257 - val_loss: 1.3343 - val_accuracy: 0.5973
Epoch 122/250
19/19 [==============================] - 0s 5ms/step - loss: 0.3194 - accuracy: 0.9291 - val_loss: 1.3095 - val_accuracy: 0.5973
Epoch 123/250

19/19 [==============================] - 0s 5ms/step - loss: 0.1701 - accuracy: 0.9713 - val_loss: 1.4611 - val_accuracy: 0.5906
Epoch 174/250
19/19 [==============================] - 0s 4ms/step - loss: 0.1652 - accuracy: 0.9662 - val_loss: 1.4670 - val_accuracy: 0.5973
Epoch 175/250
19/19 [==============================] - 0s 4ms/step - loss: 0.1651 - accuracy: 0.9696 - val_loss: 1.4763 - val_accuracy: 0.5906
Epoch 176/250
19/19 [==============================] - 0s 5ms/step - loss: 0.1616 - accuracy: 0.9764 - val_loss: 1.4769 - val_accuracy: 0.5906
Epoch 177/250
19/19 [==============================] - 0s 6ms/step - loss: 0.1607 - accuracy: 0.9747 - val_loss: 1.4736 - val_accuracy: 0.5906
Epoch 178/250
19/19 [==============================] - 0s 5ms/step - loss: 0.1586 - accuracy: 0.9713 - val_loss: 1.4723 - val_accuracy: 0.5906
Epoch 179/250
19/19 [==============================] - 0s 4ms/step - loss: 0.1561 - accuracy: 0.9696 - val_loss: 1.4867 - val_accuracy: 0.5906
Epoch 180/250

19/19 [==============================] - 0s 7ms/step - loss: 0.0833 - accuracy: 0.9916 - val_loss: 1.6678 - val_accuracy: 0.5906
Epoch 231/250
19/19 [==============================] - 0s 6ms/step - loss: 0.0803 - accuracy: 0.9916 - val_loss: 1.6883 - val_accuracy: 0.5906
Epoch 232/250
19/19 [==============================] - 0s 6ms/step - loss: 0.0807 - accuracy: 0.9916 - val_loss: 1.6958 - val_accuracy: 0.5906
Epoch 233/250
19/19 [==============================] - 0s 7ms/step - loss: 0.0787 - accuracy: 0.9916 - val_loss: 1.7207 - val_accuracy: 0.5973
Epoch 234/250
19/19 [==============================] - 0s 7ms/step - loss: 0.0790 - accuracy: 0.9966 - val_loss: 1.6972 - val_accuracy: 0.5906
Epoch 235/250
19/19 [==============================] - 0s 5ms/step - loss: 0.0775 - accuracy: 0.9932 - val_loss: 1.7188 - val_accuracy: 0.5973
Epoch 236/250
19/19 [==============================] - 0s 6ms/step - loss: 0.0765 - accuracy: 0.9932 - val_loss: 1.7187 - val_accuracy: 0.5973
Epoch 237/250

In [10]:
loss,acc = model.evaluate(X_test_pca,Y_test_encoded)
loss,acc

10/10 [==============================] - 0s 2ms/step - loss: 1.9030 - accuracy: 0.6384


(1.9030320644378662, 0.6383647918701172)